# Simulation template 

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963). Here the simulation is run with features generated from fourier components in the first scenario. This scenario is characterized by using only the EVEN wavevector space eigenmodes, computed from ALL real space components.

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 761 ms, sys: 281 ms, total: 1.04 s
Wall time: 609 ms


In [2]:
%%time
### Dataset and simulation parameters
#csv_path = SSH2_PERIODIC_100_6561_CSV 
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
#allowed_windings = [-1,0,1,2]
simulation_dir = SSH1_REAL_SPACE_ALL_SITES
#simulation_dir = SSH2_PERIODIC_ENGINEERED_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None
shuffle_features = False
random_state = 763

### Fourier features
fourier_mode = "dst"
fourier_features_to_use = None
fourier_real = "imag"
fourier_normalize = False
fourier_fillna = False

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
#mode=mode
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_100_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = (
    "/home/rio/ssh_simulations/ssh2/periodic_100_6561/dct_best_12_wavevector_sites"
)
random_state = 2016
fourier_features_to_use = [0, 1, 2, 3, 4, 5, 6, 7, 47, 48, 49, 50]
fourier_mode = "dct"
fourier_real = None
fourier_normalize = False
fourier_fillna = False


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,fourier_mode,fourier_features_to_use,fourier_real,\
                       fourier_normalize, fourier_fillna , random_state)
#simulation = Simulation(**dict_args)


#(self, csv_path, model_name, model_kw, allowed_windings, simulation_dir = None, val_split = 0, 
# features_to_use = None, shuffle_features = False, fourier_mode=None, fourier_features_to_use=None,
# fourier_real = False, fourier_normalize = False, Fourier_fillna=False, random_state = None):



print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Columns: 108 entries, id to feat99
dtypes: float64(103), int32(3), object(2)
memory usage: 538.1+ MB
CPU times: user 5.61 s, sys: 582 ms, total: 6.19 s
Wall time: 5.5 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130965,0.301130,...,-0.080279,-0.115687,-0.174988,0.245064,0.319763,-0.059082,-0.000556,-0.245133,-0.301130,0.130965
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130965,-0.301130,...,-0.080279,0.115687,-0.174988,-0.245064,0.319763,0.059082,-0.000556,0.245133,-0.301130,-0.130965
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.148414,-0.042467,...,0.185442,-0.080620,-0.066211,-0.121339,-0.151700,0.192513,0.191642,-0.050956,-0.042467,-0.148414
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.069630,-0.026147,...,0.102009,-0.100671,-0.081002,-0.020715,-0.058717,0.115834,0.118274,-0.054666,-0.026147,-0.069630
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.010206,-0.014625,...,0.047555,-0.041368,-0.018852,-0.017673,-0.023377,0.031447,0.010280,0.000925,0.014625,-0.010206


####  Fourier features

In [5]:
%%time
#simulation.generate_fourier_df()
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.info() 
    simulation.fourier_dataframe.sample(15)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Data columns (total 20 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          656100 non-null  int32  
 1   path        656100 non-null  object 
 2   t1          656100 non-null  float64
 3   t2          656100 non-null  float64
 4   winding     656100 non-null  float64
 5   phase       656100 non-null  int32  
 6   pred_phase  656100 non-null  int32  
 7   type_of     656100 non-null  object 
 8   dct_feat0   656100 non-null  float64
 9   dct_feat1   656100 non-null  float64
 10  dct_feat2   656100 non-null  float64
 11  dct_feat3   656100 non-null  float64
 12  dct_feat4   656100 non-null  float64
 13  dct_feat5   656100 non-null  float64
 14  dct_feat6   656100 non-null  float64
 15  dct_feat7   656100 non-null  float64
 16  dct_feat47  656100 non-null  float64
 17  dct_feat48  656100 non-null  float64
 18  dct_feat49  656100 non-null  float64
 19  dc

In [6]:
print("features_to_use: ")
print("\n")
print(simulation.features_to_use)
if simulation.features_to_use is not None:
    print("Number of real space features:")
    print("\n")
    print(len(simulation.features_to_use))
print("\n")
print("fourier_features: ")
print("\n")
print(simulation.fourier_features)
print("\n")
if simulation.fourier_features is not None:
    print("Number of Fourier features: ")
    print("\n")
    print(len(simulation.fourier_features))
print("\n")
print("Fourier operator: ")
print(simulation.fourier_operator)
print("\n")
if simulation.fourier_operator is not None:
    print("Shape of Fourier operator: ")
    print(simulation.fourier_operator.shape)

features_to_use: 


None


fourier_features: 


['dct_feat0', 'dct_feat1', 'dct_feat2', 'dct_feat3', 'dct_feat4', 'dct_feat5', 'dct_feat6', 'dct_feat7', 'dct_feat47', 'dct_feat48', 'dct_feat49', 'dct_feat50']


Number of Fourier features: 


12


Fourier operator: 
[[ 1.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00
   2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00
   2.00000000e+00  2.00000000e+00  2.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.99605346e+00  1.98422940e+00  1.96457450e+00
   1.93716632e+00  1.90211303e+00  1.85955297e+00  1.80965410e+00
  -1.96457450e+00 -1.98422940e+00 -1.99605346e+00 -1.00000000e+00]
 [ 1.00000000e+00  1.98422940e+00  1.93716632e+00  1.85955297e+00
   1.75261336e+00  1.61803399e+00  1.45793725e+00  1.27484798e+00
   1.85955297e+00  1.93716632e+00  1.98422940e+00  1.00000000e+00]
 [ 1.00000000e+00  1.96457450e+00  1.85955297e+00  1.68865585e+00
   1.45793725e+00  1.17557050e+00  8.51558583e-01  4.97379774e-01
  -1.

In [7]:
#a = np.array([1+1j,2+3j,4+8j])
#np.imag(a)

In [8]:
#plt.plot(np.imag(simulation.fourier_operator[:,0]))

In [9]:
#np.sin(np.pi/(50))*2

In [10]:
#2*np.sin(np.pi/50*49)

In [11]:
#simulation.dataframe.iloc[331456,:]

In [12]:
#for ix, obj in enumerate(simulation.dataframe.iloc[331456,:].values):
#    print(ix,obj)

In [13]:
#simulation.fourier_engineered_dataframe.dct_feat0

In [14]:
#simulation.features

In [15]:
#simulation.fourier_matrix

In [16]:
#1/simulation.n_features*simulation.dataframe.loc[:,simulation.features].sum(axis=1)

In [17]:
#df_2 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==2]
#df_2.describe()

In [18]:
#df_1 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==1]
#df_1.describe()

In [19]:
#df_0 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==0]
#df_0.describe()

In [20]:
#plt.figure(figsize=(10,10))
#plt.scatter(df_1.fourier_feat0,df_1.fourier_feat2)
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [21]:
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [22]:
#plt.scatter(df_2.fourier_feat0,df_2.fourier_feat2)

In [23]:
#df_1.hist(figsize=(15,15))

In [24]:
#df_0.hist(figsize=(15,15))

#### Checking initialization

In [25]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  100
n_hamiltonians:  6561
n_ts:  2
CPU times: user 191 µs, sys: 26 µs, total: 217 µs
Wall time: 120 µs


In [26]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [27]:

%%time
simulation.run_simulation(n_experiments, start_n, fit_params,shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

Simulation mode:  dct


running experiments:   1%|          | 1/100 [00:19<32:25, 19.65s/it]

running experiments:   2%|▏         | 2/100 [00:41<32:56, 20.17s/it]

running experiments:   3%|▎         | 3/100 [01:02<33:18, 20.60s/it]

...

running experiments:  98%|█████████▊| 98/100 [33:23<00:39, 19.95s/it]

running experiments:  99%|█████████▉| 99/100 [33:43<00:19, 19.88s/it]

running experiments: 100%|██████████| 100/100 [34:03<00:00, 19.90s/it]

running experiments: 100%|██████████| 100/100 [34:03<00:00, 20.43s/it]

CPU times: user 3h 43min 39s, sys: 9.64 s, total: 3h 43min 49s
Wall time: 34min 3s


In [28]:
simulation.dataframe.head(10)

,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130965,0.301130,...,-0.080279,-0.115687,-0.174988,0.245064,0.319763,-0.059082,-0.000556,-0.245133,-0.301130,0.130965
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130965,-0.301130,...,-0.080279,0.115687,-0.174988,-0.245064,0.319763,0.059082,-0.000556,0.245133,-0.301130,-0.130965
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.148414,-0.042467,...,0.185442,-0.080620,-0.066211,-0.121339,-0.151700,0.192513,0.191642,-0.050956,-0.042467,-0.148414
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.069630,-0.026147,...,0.102009,-0.100671,-0.081002,-0.020715,-0.058717,0.115834,0.118274,-0.054666,-0.026147,-0.069630
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.010206,-0.014625,...,0.047555,-0.041368,-0.018852,-0.017673,-0.023377,0.031447,0.010280,0.000925,0.014625,-0.010206
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.116962,0.021475,...,-0.086901,-0.045676,-0.028034,0.129266,0.130083,-0.098190,-0.104188,-0.007911,0.021475,0.116962
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.037175,-0.095882,...,-0.027490,0.046168,0.063419,-0.041630,-0.085769,-0.010604,-0.009822,0.084024,0.095882,-0.037175
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.027709,-0.038180,...,0.123344,-0.114320,-0.057118,-0.040291,-0.061263,0.087153,0.029599,-0.001276,0.038180,-0.027709
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.059795,0.030272,...,-0.082347,0.140996,0.128386,-0.036608,0.029628,-0.124877,-0.140478,0.088573,0.030272,0.059795
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.141605,0.013497,...,-0.035718,-0.110014,-0.082452,0.146084,0.152588,-0.067575,-0.087187,-0.043484,0.013497,0.141605


#### Viewing a random sample

In [29]:
%%time
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.sample(frac=0.1, replace=False)

CPU times: user 15.8 ms, sys: 3.99 ms, total: 19.8 ms
Wall time: 19.6 ms


#### Checking train/val/test splits again

In [30]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [596, 445, 4413, 3350, 5312, 4746, 5279, 4506, 6082, 5588, 5522, 3541, 3628, 3871, 4819, 5678, 5107, 2821, 5621, 3142, 2495, 6092, 6133, 4294, 2649, 3693, 1783, 5469, 2853, 2748, 5537, 1762, 1349, 2749, 2529, 4655, 1843, 2995, 346, 2191, 1182, 5144, 3817, 3386, 4195, 4201, 4089, 5806, 3951, 4868, 3870, 1204, 2404, 156, 2637, 2320, 5756, 4815, 1704, 3339, 4299, 3359, 3427, 3498, 357, 2434, 126, 2659, 2779, 4067, 3677, 4863, 2857, 1411, 3243, 805, 5723, 3452, 5719, 832, 3179, 4017, 3189, 4768, 4548, 3276, 6345, 2085, 3314, 5918, 879, 5526, 2393, 547, 4577, 1994, 2176, 3969, 3504, 400, 1444, 616, 125, 151, 5243, 2406, 3696, 1676, 1282, 366, 2879, 6005, 4896, 628, 3791, 4642, 6121, 4561, 5304, 1745, 3862, 2499, 6341, 757, 32

#### Checking summaries

In [31]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 82 µs, sys: 0 ns, total: 82 µs
Wall time: 58.2 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.05,0.41,0.27,0.27,999,0
1,-2.0,-1.95,test,0.11,0.34,0.23,0.32,-1,0
2,-2.0,-1.90,test,0.10,0.37,0.23,0.30,-1,0
3,-2.0,-1.85,test,0.12,0.39,0.22,0.27,-1,0
4,-2.0,-1.80,test,0.08,0.40,0.27,0.25,-1,0
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.30,0.20,0.20,0.30,-1,2
6557,2.0,1.85,test,0.26,0.18,0.20,0.36,-1,2
6558,2.0,1.90,test,0.17,0.22,0.19,0.42,-1,2


In [32]:
ham_summary.describe()

,t1,t2,-1,0,1,2,phase,pred_phase
count,6561.000000,6.561000e+03,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000
mean,0.000000,-4.331917e-18,0.187793,0.318749,0.312355,0.181103,28.045877,0.499771
std,1.169134,1.169134e+00,0.337620,0.417621,0.414506,0.330360,163.557011,1.010482
min,-2.000000,-2.000000e+00,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,-1.000000,-1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000e+00,0.010000,0.050000,0.050000,0.000000,1.000000,0.000000
75%,1.000000,1.000000e+00,0.130000,0.900000,0.870000,0.120000,1.000000,1.000000
max,2.000000,2.000000e+00,1.000000,1.000000,1.000000,1.000000,999.000000,2.000000


In [33]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  656100


,id,phase,pred_phase,type_of
0,0,999,-1,test
1,0,999,0,test
2,0,999,1,test
3,0,999,1,test
4,0,999,1,test
...,...,...,...,...
656095,6560,999,0,test
656096,6560,999,0,test
656097,6560,999,-1,test
656098,6560,999,-1,test


#### Checking accuracies

In [34]:
simulation.accuracy

{'eigenvector_train': 0.9993408185440058,
 'eigenvector_val': 0.8992210144927536,
 'eigenvector_test': 0.5362747380675204,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.9974637681159421,
 'hamiltonian_test': 0.7951105937136205}

#### Checking data stored in  memory

In [35]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [36]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [37]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}

In [38]:
#simulation.fourier_matrix

In [39]:
#simulation.fourier_matrix.shape

In [40]:
#simulation.fourier_matrix[:,0]

In [41]:
#simulation.fourier_matrix[:,1]

In [42]:
#np.exp(-1j*2*np.pi*3/100)

In [43]:
#for i in range(1,50):
#    print("\n")
#    print(i)
#    print(np.sum((np.real(simulation.fourier_matrix[:,i]) - np.real(simulation.fourier_matrix[:,-i]))**2))
#    print(np.sum((np.imag(simulation.fourier_matrix[:,i])+ np.imag(simulation.fourier_matrix[:,-i]))**2))

In [44]:
#print(simulation.fourier_matrix[:,0])

In [45]:
#print(simulation.fourier_matrix[:,50])